In [ ]:
# Bloco 1: importações
import pandas as pd
from pathlib import Path

try:
    NB_DIR = Path(__file__).parent.resolve()
except NameError:
    NB_DIR = Path().resolve()  # Jupyter

REPO_ROOT = NB_DIR.parent
IN_DIR = (REPO_ROOT / "data").resolve()
OUT_DIR = (REPO_ROOT / "outputs").resolve()

IN_DIR.mkdir(exist_ok=True)
OUT_DIR.mkdir(exist_ok=True)

print("IN_DIR =", IN_DIR)
print("OUT_DIR =", OUT_DIR)


In [ ]:
def read_csv_robusto(caminho, **kwargs):
    """Le CSV tentando várias codificações."""
    encodings = ["utf-8", "latin-1", "utf-16", "utf-8-sig"]
    for enc in encodings:
        try:
            return pd.read_csv(caminho, encoding=enc, **kwargs)
        except UnicodeDecodeError:
            continue
    with open(caminho, 'r', errors='replace') as f:
        return pd.read_csv(f, **kwargs)


In [ ]:
def criar_isolados_detalhados(df):
    """Tabela de isolados com nomes originais."""
    cols = {
        "dthr_liberada": "Data",
        "codigo_paciente": "Código Paciente",
        "paciente": "Paciente",
        "unidade_solicitante": "Unidade",
        "descmaterial_analise": "Tipo de Amostra",
        "microorganismo": "Microrganismo",
        "Antibiótico": "Antimicrobiano",
        "RSI": "Resultado"
    }
    disponiveis = [c for c in cols if c in df.columns]
    return df[disponiveis].rename(columns=cols)


In [ ]:
def criar_sensibilidade(df):
    """Agrega resultados de sensibilidade por microrganismo/antimicrobiano."""
    grupo = (
        df.groupby(["microorganismo", "Antibiótico", "RSI"]).size()
          .unstack(fill_value=0)
    )
    total = grupo.sum(axis=1)
    pct = grupo.div(total, axis=0) * 100
    grupo.columns = [f"{c}_n" for c in grupo.columns]
    pct.columns = [f"{c}_pct" for c in pct.columns]
    tabela = pd.concat([grupo, pct], axis=1).reset_index()
    return tabela


In [ ]:
def criar_tendencia(df):
    """Resumo mensal de isolados e resistência."""
    df = df.copy()
    data = pd.to_datetime(df["dthr_liberada"])
    df["Ano"] = data.dt.year
    df["Mes"] = data.dt.month
    total = (
        df.groupby(["Ano", "Mes", "microorganismo"])["solicitacao"]
          .nunique()
          .rename("total_isolados")
    )
    resistentes = (
        df[df["RSI"] == "R"]
        .groupby(["Ano", "Mes", "microorganismo"])["solicitacao"]
        .nunique()
        .rename("resistentes")
    )
    tendencia = total.to_frame().join(resistentes, how="left").fillna(0)
    tendencia["pct_resistentes"] = (
        tendencia["resistentes"] / tendencia["total_isolados"] * 100
    )
    return tendencia.reset_index()


In [ ]:
def criar_indicadores(df, cont_pos, cont_neg):
    """Calcula taxas de incidência unindo contagens de pacientes."""
    pos = cont_pos.rename(columns=str.lower)
    neg = cont_neg.rename(columns=str.lower)
    base = pos.merge(neg, on="unidade_solicitante", how="outer", suffixes=("_pos", "_neg")).fillna(0)
    base["total_amostras"] = base.filter(regex="_pos|_neg").sum(axis=1)
    isolados = (
        df.groupby(["unidade_solicitante", "microorganismo"])["codigo_paciente"]
          .nunique()
          .rename("pacientes_positivos")
          .reset_index()
    )
    indicadores = isolados.merge(base, on="unidade_solicitante", how="left")
    indicadores["taxa_inc"] = (
        indicadores["pacientes_positivos"] / indicadores["total_amostras"] * 1000
    )
    return indicadores


In [ ]:
def gerar_excel_cciras(nome_base, df_vigiram, cont_pos, cont_neg):
    """Gera arquivo Excel com as quatro abas para o CCIRAS."""
    detalhado = criar_isolados_detalhados(df_vigiram)
    sensibilidade = criar_sensibilidade(df_vigiram)
    tendencia = criar_tendencia(df_vigiram)
    indicadores = criar_indicadores(df_vigiram, cont_pos, cont_neg)

    path_out = OUT_DIR / f"cciras-{nome_base}.xlsx"
    with pd.ExcelWriter(path_out) as writer:
        detalhado.to_excel(writer, index=False, sheet_name="Isolados Detalhados")
        sensibilidade.to_excel(writer, index=False, sheet_name="Sensibilidade")
        tendencia.to_excel(writer, index=False, sheet_name="Tendência Temporal")
        indicadores.to_excel(writer, index=False, sheet_name="Indicadores Epidemiológicos")
    print("✅ Arquivo gerado:", path_out)


In [ ]:
def processar_todos_arquivos_ano(ano: str):
    """Processa todos os vigiram-*.csv de determinado ano."""
    cont_pos = read_csv_robusto(IN_DIR / "Contagem pacientes amostra positiva.csv", dtype=str)
    cont_neg = read_csv_robusto(IN_DIR / "Contagem pacientes amostras negativas.csv", dtype=str)
    padrao = f"vigiram-*{ano}.csv"
    for csv_path in sorted(IN_DIR.glob(padrao)):
        nome_base = csv_path.stem.replace("vigiram-", "")
        print(f"🔄 Processando {nome_base}...")
        df = read_csv_robusto(csv_path, dtype=str)
        gerar_excel_cciras(nome_base, df, cont_pos, cont_neg)


In [ ]:
processar_todos_arquivos_ano("25")